# Determination of polarization

```{autolink-concat}
```

Given the aligned polarimeter field $\vec\alpha$ and the corresponding intensity distribution $I_0$, the intensity distribution $I$ for a polarized decay can be computed as follows:

$$
I\left(\phi,\theta,\chi; \tau\right) = I_0(\tau)\left(1+\vec{P} R(\phi,\theta,\chi) \vec{\alpha}(\tau)\right)
$$ (eq:master.intensity)

with $R$ the rotation matrix over the decay plane orientation, represented in Euler angles $\left(\phi, \theta, \chi\right)$.

In this section, we show that it's possible to determine the polarization $\vec{P}$ from a given intensity distribution $I$ of a $\lambda_c$&nbsp;decay if we the $\vec\alpha$&nbsp;fields and the corresponding $I_0$&nbsp;values of that $\Lambda_c$&nbsp;decay. In this example, the intensity and alpha functions are computed from exact amplitude models, but they can be substituted by functions that interpolate the exported polarimeter fields from {ref}`uncertainties:Exported distributions` over a phase space sample. This would remove the need to formulate a complete amplitude model in order to determine polarization. See {ref}`appendix/serialization:Import and interpolate` for how to interpolate these exported distributions over the Dalitz plane.

In [ ]:
from __future__ import annotations

import logging
import os
from warnings import filterwarnings

import iminuit
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import yaml
from IPython.display import Latex, Markdown
from tensorwaves.interface import DataSample, Function
from tqdm.auto import tqdm

from polarimetry import formulate_polarimetry
from polarimetry.amplitude import AmplitudeModel, DalitzPlotDecompositionBuilder
from polarimetry.data import create_data_transformer, generate_phasespace_sample
from polarimetry.io import (
    mute_jax_warnings,
    perform_cached_doit,
    perform_cached_lambdify,
)
from polarimetry.lhcb import load_model_builder, load_model_parameters
from polarimetry.lhcb.particle import load_particles
from polarimetry.plot import use_mpl_latex_fonts

filterwarnings("ignore")
mute_jax_warnings()
logging.getLogger("tensorwaves.data").setLevel(logging.ERROR)

NO_TQDM = "EXECUTE_NB" in os.environ
if NO_TQDM:
    logging.getLogger().setLevel(logging.ERROR)
    logging.getLogger("polarimetry.io").setLevel(logging.ERROR)

In [ ]:
def load_model(
    file: str, model_choice: int | str = 0, reference_subsystem: int = 1
) -> tuple[AmplitudeModel, DalitzPlotDecompositionBuilder]:
    particles = load_particles("../../data/particle-definitions.yaml")
    builder = load_model_builder(file, particles, model_choice)
    parameters = load_model_parameters(file, builder.decay, model_choice, particles)
    model = builder.formulate(reference_subsystem, cleanup_summations=True)
    model.parameter_defaults.update(parameters)
    return model, builder


MODEL_FILE = "../../data/model-definitions.yaml"
with open(MODEL_FILE) as f:
    ALL_MODEL_TITLES = list(yaml.safe_load(f))
NOMINAL_MODEL_TITLE = ALL_MODEL_TITLES[0]

MODELS = {}
INTENSITY_EXPRS = {}
POLARIMETRY_EXPRS = {}
for title in tqdm(ALL_MODEL_TITLES, desc="Formulating models", disable=NO_TQDM):
    if "LS couplings" in title:
        continue
    MODELS[title], builder = load_model(MODEL_FILE, model_choice=title)
    folded_polarimetry_exprs = formulate_polarimetry(builder, reference_subsystem=1)
    POLARIMETRY_EXPRS[title] = tuple(
        perform_cached_doit(expr.doit().xreplace(MODELS[title].amplitudes))
        for expr in tqdm(folded_polarimetry_exprs, disable=NO_TQDM, leave=False)
    )
    INTENSITY_EXPRS[title] = perform_cached_doit(MODELS[title].full_expression)
    del title, builder, folded_polarimetry_exprs

assert len(MODELS) == 17
assert len(INTENSITY_EXPRS) == 17
assert len(POLARIMETRY_EXPRS) == 17

In [ ]:
POLARIMETRY_FUNCS = {
    title: tuple(
        perform_cached_lambdify(expr.xreplace(MODELS[title].parameter_defaults))
        for expr in tqdm(polarimetry_exprs, disable=NO_TQDM, leave=False)
    )
    for title, polarimetry_exprs in tqdm(POLARIMETRY_EXPRS.items(), disable=NO_TQDM)
}
INTENSITY_FUNCS = {
    title: perform_cached_lambdify(expr.xreplace(MODELS[title].parameter_defaults))
    for title, expr in tqdm(INTENSITY_EXPRS.items(), disable=NO_TQDM)
}

NOMINAL_MODEL_TITLE = ALL_MODEL_TITLES[0]
NOMINAL_MODEL = MODELS[NOMINAL_MODEL_TITLE]
NOMINAL_INTENSITY_FUNC = INTENSITY_FUNCS[NOMINAL_MODEL_TITLE]
NOMINAL_POLARIMETRY_FUNCS = POLARIMETRY_FUNCS[NOMINAL_MODEL_TITLE]

The phase space sample is uniformly generated over the Dalitz plane variables $\tau$. In addition, we extend the phase space samples with a uniform distribution over the decay plane angles $\left(\phi, \theta, \chi\right)$.

In [ ]:
N_EVENTS = 100_000
# Dalitz variables
PHSP = generate_phasespace_sample(NOMINAL_MODEL.decay, N_EVENTS, seed=0)
PHSP.update(create_data_transformer(NOMINAL_MODEL)(PHSP))
# Decay plane variables
PHSP["phi"] = np.random.uniform(-np.pi, +np.pi, N_EVENTS)
PHSP["cos_theta"] = np.random.uniform(-1, +1, N_EVENTS)
PHSP["chi"] = np.random.uniform(-np.pi, +np.pi, N_EVENTS)

We now generate an intensity distribution over the phase space sample given a certain[^1] value for $\vec{P}$ using Eq.&nbsp;{eq}`eq:master.intensity`:

[^1]: See [p.&nbsp;18](https://arxiv.org/pdf/2208.03262.pdf#page=20) of _Amplitude analysis of the $\Lambda^+_c \to p K^- \pi^+$ decay and $\Lambda^+_c$ baryon polarization measurement in semileptonic beauty hadron decays_ (2022) [[link]](https://inspirehep.net/literature/2132745)

In [ ]:
def compute_polarized_intensity(
    Px: float,
    Py: float,
    Pz: float,
    intensity_func: Function[DataSample, jnp.ndarray],
    polarimetry_funcs: tuple[
        Function[DataSample, jnp.ndarray],
        Function[DataSample, jnp.ndarray],
        Function[DataSample, jnp.ndarray],
    ],
    phsp: DataSample,
) -> jnp.array:
    P = jnp.array([Px, Py, Pz])
    R = compute_rotation_matrix(phsp)
    alpha = jnp.array([func(phsp).real for func in polarimetry_funcs])
    I0 = intensity_func(phsp)
    return I0 * (1 + jnp.einsum("i,ij...,j...->...", P, R, alpha))


def compute_rotation_matrix(phsp: DataSample) -> jnp.ndarray:
    ϕ = phsp["phi"]
    θ = jnp.arccos(phsp["cos_theta"])
    χ = phsp["chi"]
    return jnp.einsum("ki...,ij...,j...->k...", Rz(ϕ), Ry(θ), Rz(χ))


def Rz(angle: jnp.ndarray) -> jnp.ndarray:
    n_events = len(angle)
    ones = jnp.ones(n_events)
    zeros = jnp.zeros(n_events)
    return jnp.array(
        [
            [+jnp.cos(angle), -jnp.sin(angle), zeros],
            [+jnp.sin(angle), +jnp.cos(angle), zeros],
            [zeros, zeros, ones],
        ]
    )


def Ry(angle: jnp.ndarray) -> jnp.ndarray:
    n_events = len(angle)
    ones = jnp.ones(n_events)
    zeros = jnp.zeros(n_events)
    return jnp.array(
        [
            [+jnp.cos(angle), zeros, +jnp.sin(angle)],
            [zeros, ones, zeros],
            [-jnp.sin(angle), zeros, +jnp.cos(angle)],
        ]
    )

In [ ]:
P = (+0.2165, +0.0108, -0.665)
I = compute_polarized_intensity(
    *P, NOMINAL_INTENSITY_FUNC, NOMINAL_POLARIMETRY_FUNCS, PHSP
)
I /= jnp.mean(I)  # normalized times N for log likelihood

In [ ]:
plt.rc("font", size=18)
use_mpl_latex_fonts()

fig, axes = plt.subplots(figsize=(15, 4), ncols=3)
fig.tight_layout()
for ax in axes:
    ax.set_yticks([])
axes[0].hist(PHSP["phi"], weights=I, bins=80)
axes[1].hist(PHSP["cos_theta"], weights=I, bins=80)
axes[2].hist(PHSP["chi"], weights=I, bins=80)
axes[0].set_xlabel(R"$\phi$")
axes[1].set_xlabel(R"$\cos\theta$")
axes[2].set_xlabel(R"$\chi$")
plt.show()

fig, ax = plt.subplots(figsize=(12, 3))
ax.hist2d(PHSP["sigma2"], PHSP["sigma1"], weights=I, bins=100, cmin=1)
ax.set_xlabel(R"$\sigma_2$")
ax.set_ylabel(R"$\sigma_1$")
ax.set_aspect("equal")
plt.show()

The generated distribution is now assumed to be a _measured distribution_&nbsp;$I$ with unknown polarization&nbsp;$\vec{P}$. It is shown below that the actual $\vec{P}$ with which the distribution was generated can be found by performing a fit on Eq.&nbsp;{eq}`eq:master.intensity`. This is done with [`iminuit`](https://iminuit.rtfd.io), starting with a certain 'guessed' value for $\vec{P}$ as initial parameters.

To avoid having to generate a hit-and-miss intensity test distribution, the parameters $\vec{P} = \left(P_x, P_y, P_z\right)$ are optimized with regard to a **weighted negative log likelihood estimator**:

$$
\mathrm{NLL} = -\sum_i w_i \log I_{i,\vec{P}}\left(\phi,\theta,\chi;\tau\right)\,.
$$ (eq:weighted-nll)

with the normalized intensities of the generated distribution taken as weights:

$$
w_i = n\,I_i\,\big/\,\sum_j^n I_j\,,
$$ (eq:intensity-as-nll-weight)

such that $\sum w_i = n$.

In [ ]:
P_guess = (+0.3, -0.3, +0.4)

In [ ]:
def perform_fit(model_title: str) -> iminuit.Minuit:
    progress_bar = tqdm(desc=model_title, disable=NO_TQDM, leave=False)
    intensity_func = INTENSITY_FUNCS[model_title]
    polarimetry_funcs = POLARIMETRY_FUNCS[model_title]

    def weighted_nll(Px: float, Py: float, Pz: float) -> float:
        I_new = compute_polarized_intensity(
            Px, Py, Pz, intensity_func, polarimetry_funcs, PHSP
        )
        I_new /= jnp.sum(I_new)
        progress_bar.update()
        estimator_value = -jnp.sum(jnp.log(I_new) * I)
        progress_bar.set_postfix_str(
            f"NLL={estimator_value:.4g}, Px={Px:+.3f}, Py={Py:+.3f}, Pz={Pz:+.3f}"
        )
        return estimator_value

    optimizer = iminuit.Minuit(weighted_nll, *P_guess)
    progress_bar.close()
    optimizer.errordef = optimizer.LIKELIHOOD
    return optimizer.migrad()


FIT_RESULTS = {
    title: perform_fit(title)
    for title in tqdm(MODELS, desc="Performing fit", disable=NO_TQDM)
}

In [ ]:
P_fit_values = 100 * np.array(
    [[p.value for p in fit_result.params] for fit_result in FIT_RESULTS.values()]
)
P_fit_nominal = P_fit_values[0]
P_max = (P_fit_values[1:] - P_fit_nominal).max(axis=0)
P_min = (P_fit_values[1:] - P_fit_nominal).min(axis=0)
np.testing.assert_array_almost_equal(P_fit_nominal, 100 * np.array(P), decimal=3)


def render_p(i: int) -> str:
    return f"{P_fit_nominal[i]:+.2f}_{{{P_min[i]:+.2f}}}^{{{P_max[i]:+.2f}}}"


src = Rf"""
The polarization $\vec{{P}}$ is determined to be (in %):

$$
\begin{{array}}{{ccc}}
P_x &=& {render_p(0)} \\
P_y &=& {render_p(1)} \\
P_z &=& {render_p(2)} \\
\end{{array}}
$$

with the upper and lower sign being the systematic extrema uncertainties as determined by
the alternative models.
"""
Markdown(src)

This is to be compared with the model uncertainties reported by [^1]:

$$
\begin{array}{ccc}
P_x &=& +21.65 \pm 0.36 \\
P_y &=&  +1.08 \pm 0.09 \\
P_z &=& -66.5 \pm 1.1. \\
\end{array}
$$

The polarimeter values for each model are (in %):

In [ ]:
src = R"""
\begin{array}{r|ccc|ccc}
  \textbf{Model} & \mathbf{P_x} & \mathbf{P_y} & \mathbf{P_z}
  & \mathbf{\Delta P_x} & \mathbf{\Delta P_y} & \mathbf{\Delta P_z} \\
  \hline
"""
Px_nom, Py_nom, Pz_nom = P_fit_nominal
for i, (Px, Py, Pz) in enumerate(P_fit_values):
    src += Rf"  \textbf{{{i}}} & {Px:+.2f} & {Py:+.2f} & {Pz:+.1f} & "
    if i != 0:
        src += Rf"{Px-Px_nom:+.2f} & {Py-Py_nom:+.2f} & {Pz-Pz_nom:+.2f}"
    src += R" \\" "\n"
    del Px, Py, Pz
del Px_nom, Py_nom, Pz_nom
src += R"\end{array}"
Latex(src)